# E-Commerce Sales Intelligence Dashboard

## Interactive Analytics Platform

This dashboard provides comprehensive insights into e-commerce performance with:
- **Real-time filtering** by region, category, and date range
- **Advanced SQL analytics** including CLV and window functions
- **Dynamic visualizations** that update based on user selections
- **Performance metrics** and trend analysis

---

In [1]:
# ========================================
# SECTION 1: IMPORTS AND CONFIGURATION
# ========================================

import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Interactive widgets for user controls
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML, clear_output

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Set default plotly theme
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✓ All libraries imported successfully")
print("✓ Dashboard initialized")

✓ All libraries imported successfully
✓ Dashboard initialized


In [ ]:
# ========================================
# SECTION 2: DATABASE CONNECTION
# ========================================

def get_database_connection():
    """
    Establish connection to the SQLite database.
    The database should be created from the analysis notebook first.

    Returns:
        sqlite3.Connection: Active database connection
    """
    # Path to the database created in the analysis folder
    db_path = '../analysis/ecommerce.db'

    try:
        conn = sqlite3.connect(db_path)
        print(f"✓ Connected to database: {db_path}")
        return conn
    except sqlite3.Error as e:
        print(f"✗ Database connection error: {e}")
        print("\n⚠️  Please run the database_manipulation.ipynb notebook in the analysis folder first to generate the database.")
        return None

# Establish connection
conn = get_database_connection()

if conn:
    # Test the connection by getting table info
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"\n✓ Found {len(tables)} tables: {', '.join([t[0] for t in tables])}")

✓ Connected to database: ../analysis/ecommerce.db

✓ Found 0 tables: 


In [ ]:
# ========================================
# SECTION 3: DATA LOADING FUNCTIONS
# ========================================

def get_filter_options(conn):
    """
    Get available filter options from the database.
    Used to populate dropdown widgets.

    Returns:
        dict: Dictionary containing lists of regions, categories, and date range
    """
    # Get unique regions
    regions = pd.read_sql_query("SELECT DISTINCT region FROM customers ORDER BY region", conn)['region'].tolist()

    # Get unique categories
    categories = pd.read_sql_query("SELECT DISTINCT category FROM products ORDER BY category", conn)['category'].tolist()

    # Get date range
    date_query = "SELECT MIN(order_date) as min_date, MAX(order_date) as max_date FROM orders"
    date_range = pd.read_sql_query(date_query, conn)

    return {
        'regions': ['All'] + regions,
        'categories': ['All'] + categories,
        'min_date': date_range['min_date'].iloc[0],
        'max_date': date_range['max_date'].iloc[0]
    }

def get_filtered_data(conn, region='All', category='All', start_date=None, end_date=None):
    """
    Retrieve data based on user-selected filters.

    Args:
        conn: Database connection
        region: Selected region (or 'All')
        category: Selected product category (or 'All')
        start_date: Start date for filtering
        end_date: End date for filtering

    Returns:
        dict: Dictionary containing filtered datasets
    """
    # Build WHERE clause based on filters
    conditions = ["o.status = 'Completed'"]

    if region != 'All':
        conditions.append(f"c.region = '{region}'")

    if category != 'All':
        conditions.append(f"p.category = '{category}'")

    if start_date:
        conditions.append(f"o.order_date >= '{start_date}'")

    if end_date:
        conditions.append(f"o.order_date <= '{end_date}'")

    where_clause = " AND ".join(conditions)

    # Query 1: Revenue by region and category (for heatmap)
    heatmap_query = f"""
        SELECT
            c.region,
            p.category,
            SUM(o.total_amount) as total_revenue,
            COUNT(o.order_id) as order_count
        FROM orders o
        INNER JOIN customers c ON o.customer_id = c.customer_id
        INNER JOIN products p ON o.product_id = p.product_id
        WHERE {where_clause}
        GROUP BY c.region, p.category
    """

    # Query 2: Daily revenue trends
    trends_query = f"""
        WITH daily_revenue AS (
            SELECT
                o.order_date,
                SUM(o.total_amount) as daily_total,
                COUNT(o.order_id) as daily_orders
            FROM orders o
            INNER JOIN customers c ON o.customer_id = c.customer_id
            INNER JOIN products p ON o.product_id = p.product_id
            WHERE {where_clause}
            GROUP BY o.order_date
        )
        SELECT
            order_date,
            daily_total,
            daily_orders,
            AVG(daily_total) OVER (
                ORDER BY order_date
                ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
            ) as moving_avg_7day
        FROM daily_revenue
        ORDER BY order_date
    """

    # Query 3: Customer Lifetime Value
    clv_query = f"""
        SELECT
            c.customer_id,
            c.name,
            c.region,
            c.customer_segment,
            COUNT(o.order_id) as total_orders,
            SUM(o.total_amount) as customer_lifetime_value,
            AVG(o.total_amount) as average_order_value
        FROM customers c
        INNER JOIN orders o ON c.customer_id = o.customer_id
        INNER JOIN products p ON o.product_id = p.product_id
        WHERE {where_clause}
        GROUP BY c.customer_id, c.name, c.region, c.customer_segment
        ORDER BY customer_lifetime_value DESC
        LIMIT 30
    """

    # Query 4: Product performance
    product_query = f"""
        SELECT
            p.product_id,
            p.product_name,
            p.category,
            p.price,
            SUM(o.quantity) as total_units_sold,
            SUM(o.total_amount) as total_revenue,
            COUNT(DISTINCT o.order_id) as total_orders,
            COALESCE(AVG(r.rating), 0) as avg_rating,
            COUNT(DISTINCT r.review_id) as review_count
        FROM products p
        INNER JOIN orders o ON p.product_id = o.product_id
        INNER JOIN customers c ON o.customer_id = c.customer_id
        LEFT JOIN reviews r ON p.product_id = r.product_id
        WHERE {where_clause}
        GROUP BY p.product_id, p.product_name, p.category, p.price
        ORDER BY total_revenue DESC
        LIMIT 20
    """

    # Query 5: Top categories by region (Window Function)
    top_category_query = f"""
        WITH regional_sales AS (
            SELECT
                c.region,
                p.category,
                SUM(o.total_amount) as total_revenue,
                COUNT(o.order_id) as total_orders,
                RANK() OVER (
                    PARTITION BY c.region
                    ORDER BY SUM(o.total_amount) DESC
                ) as category_rank
            FROM orders o
            INNER JOIN customers c ON o.customer_id = c.customer_id
            INNER JOIN products p ON o.product_id = p.product_id
            WHERE {where_clause}
            GROUP BY c.region, p.category
        )
        SELECT region, category as top_category, total_revenue, total_orders
        FROM regional_sales
        WHERE category_rank = 1
        ORDER BY total_revenue DESC
    """

    # Execute all queries
    return {
        'heatmap': pd.read_sql_query(heatmap_query, conn),
        'trends': pd.read_sql_query(trends_query, conn),
        'clv': pd.read_sql_query(clv_query, conn),
        'products': pd.read_sql_query(product_query, conn),
        'top_categories': pd.read_sql_query(top_category_query, conn)
    }

# Get available filter options
if conn:
    filter_options = get_filter_options(conn)
    print(f"\n✓ Filter options loaded:")
    print(f"  • Regions: {len(filter_options['regions'])-1}")
    print(f"  • Categories: {len(filter_options['categories'])-1}")
    print(f"  • Date range: {filter_options['min_date']} to {filter_options['max_date']}")

In [ ]:
# ========================================
# SECTION 4: VISUALIZATION FUNCTIONS
# ========================================

def create_kpi_cards(data):
    """
    Create KPI summary cards showing key metrics.

    Args:
        data: Dictionary containing filtered datasets
    """
    # Calculate KPIs
    total_revenue = data['heatmap']['total_revenue'].sum()
    total_orders = data['heatmap']['order_count'].sum()
    avg_order_value = total_revenue / total_orders if total_orders > 0 else 0
    total_customers = len(data['clv'])
    avg_clv = data['clv']['customer_lifetime_value'].mean() if len(data['clv']) > 0 else 0

    # Create HTML for KPI cards
    kpi_html = f"""
    <style>
        .kpi-container {{
            display: flex;
            justify-content: space-around;
            margin: 20px 0;
            gap: 15px;
        }}
        .kpi-card {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border-radius: 10px;
            padding: 20px;
            flex: 1;
            color: white;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }}
        .kpi-card:nth-child(2) {{
            background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
        }}
        .kpi-card:nth-child(3) {{
            background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
        }}
        .kpi-card:nth-child(4) {{
            background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%);
        }}
        .kpi-title {{
            font-size: 14px;
            opacity: 0.9;
            margin-bottom: 10px;
        }}
        .kpi-value {{
            font-size: 32px;
            font-weight: bold;
        }}
    </style>
    <div class="kpi-container">
        <div class="kpi-card">
            <div class="kpi-title">Total Revenue</div>
            <div class="kpi-value">${total_revenue:,.0f}</div>
        </div>
        <div class="kpi-card">
            <div class="kpi-title">Total Orders</div>
            <div class="kpi-value">{total_orders:,}</div>
        </div>
        <div class="kpi-card">
            <div class="kpi-title">Avg Order Value</div>
            <div class="kpi-value">${avg_order_value:,.2f}</div>
        </div>
        <div class="kpi-card">
            <div class="kpi-title">Avg CLV</div>
            <div class="kpi-value">${avg_clv:,.2f}</div>
        </div>
    </div>
    """

    display(HTML(kpi_html))

def create_regional_heatmap(data):
    """
    Create interactive heatmap of sales by region and category.
    """
    df = data['heatmap']

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    pivot_data = df.pivot_table(
        index='region',
        columns='category',
        values='total_revenue',
        fill_value=0
    )

    fig = go.Figure(data=go.Heatmap(
        z=pivot_data.values,
        x=pivot_data.columns,
        y=pivot_data.index,
        colorscale='YlOrRd',
        text=np.round(pivot_data.values, 0),
        texttemplate='$%{text:,.0f}',
        textfont={"size": 10},
        colorbar=dict(title="Revenue ($)")
    ))

    fig.update_layout(
        title='📊 Sales Density Heatmap: Revenue by Region × Category',
        xaxis_title='Product Category',
        yaxis_title='Region',
        height=400,
        font=dict(size=11)
    )

    fig.show()

def create_revenue_trends(data):
    """
    Create line chart showing daily revenue trends with moving average.
    """
    df = data['trends']

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    fig = go.Figure()

    # Daily revenue
    fig.add_trace(go.Scatter(
        x=df['order_date'],
        y=df['daily_total'],
        mode='lines',
        name='Daily Revenue',
        line=dict(color='rgba(100, 149, 237, 0.4)', width=1),
        fill='tozeroy',
        fillcolor='rgba(100, 149, 237, 0.1)'
    ))

    # 7-day moving average
    fig.add_trace(go.Scatter(
        x=df['order_date'],
        y=df['moving_avg_7day'],
        mode='lines',
        name='7-Day Moving Avg',
        line=dict(color='rgb(220, 20, 60)', width=3)
    ))

    fig.update_layout(
        title='📈 Daily Revenue Trends with 7-Day Moving Average',
        xaxis_title='Date',
        yaxis_title='Revenue ($)',
        height=400,
        hovermode='x unified',
        legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.8)')
    )

    fig.show()

def create_clv_chart(data):
    """
    Create horizontal bar chart of top customers by CLV.
    """
    df = data['clv'].head(15)

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    fig = go.Figure(data=[
        go.Bar(
            x=df['customer_lifetime_value'],
            y=df['name'],
            orientation='h',
            marker=dict(
                color=df['customer_lifetime_value'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="CLV ($)", len=0.5)
            ),
            text=df['customer_lifetime_value'].round(2),
            texttemplate='$%{text:,.2f}',
            textposition='auto',
        )
    ])

    fig.update_layout(
        title='👥 Top 15 Customers by Lifetime Value (CLV)',
        xaxis_title='Customer Lifetime Value ($)',
        yaxis_title='',
        height=500,
        yaxis=dict(autorange="reversed")
    )

    fig.show()

def create_category_pie(data):
    """
    Create pie chart showing revenue distribution by category.
    """
    df = data['heatmap'].groupby('category')['total_revenue'].sum().reset_index()
    df = df.sort_values('total_revenue', ascending=False)

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    fig = go.Figure(data=[
        go.Pie(
            labels=df['category'],
            values=df['total_revenue'],
            hole=0.4,
            textinfo='label+percent',
            textposition='auto',
            marker=dict(
                colors=px.colors.qualitative.Set3,
                line=dict(color='white', width=2)
            )
        )
    ])

    fig.update_layout(
        title='🎯 Revenue Distribution by Product Category',
        height=450,
        showlegend=True,
        legend=dict(orientation="v", yanchor="middle", y=0.5, xanchor="left", x=1.1)
    )

    fig.show()

def create_product_scatter(data):
    """
    Create scatter plot of product performance: revenue vs rating.
    """
    df = data['products']

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    fig = px.scatter(
        df,
        x='avg_rating',
        y='total_revenue',
        size='total_units_sold',
        color='category',
        hover_data=['product_name', 'review_count', 'total_orders'],
        title='⭐ Product Performance: Revenue vs Average Rating',
        labels={
            'avg_rating': 'Average Customer Rating',
            'total_revenue': 'Total Revenue ($)',
            'total_units_sold': 'Units Sold'
        },
        color_discrete_sequence=px.colors.qualitative.Bold
    )

    fig.update_layout(
        height=500,
        xaxis=dict(range=[0, 5.5]),
        showlegend=True
    )

    fig.show()

def create_top_categories_table(data):
    """
    Display table of top-selling category per region (Window Function result).
    """
    df = data['top_categories']

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    # Format the dataframe for display
    df_display = df.copy()
    df_display['total_revenue'] = df_display['total_revenue'].apply(lambda x: f"${x:,.2f}")
    df_display['total_orders'] = df_display['total_orders'].apply(lambda x: f"{x:,}")

    # Create styled HTML table
    table_html = f"""
    <style>
        .results-table {{
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .results-table th {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 12px;
            text-align: left;
        }}
        .results-table td {{
            padding: 10px 12px;
            border-bottom: 1px solid #ddd;
        }}
        .results-table tr:hover {{
            background-color: #f5f5f5;
        }}
    </style>
    <h3>🏆 Top-Selling Category per Region (Window Function Analysis)</h3>
    {df_display.to_html(classes='results-table', index=False)}
    """

    display(HTML(table_html))

print("✓ Visualization functions loaded")

In [ ]:
# ========================================
# SECTION 5: INTERACTIVE DASHBOARD
# ========================================

def create_dashboard(region, category, date_range):
    """
    Main dashboard function that updates all visualizations based on filters.

    Args:
        region: Selected region filter
        category: Selected category filter
        date_range: Tuple of (start_date, end_date)
    """
    # Clear previous output
    clear_output(wait=True)

    # Display header
    header_html = """
    <style>
        .dashboard-header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 10px;
            margin-bottom: 20px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        .dashboard-title {
            font-size: 32px;
            font-weight: bold;
            margin-bottom: 10px;
        }
        .dashboard-subtitle {
            font-size: 16px;
            opacity: 0.9;
        }
    </style>
    <div class="dashboard-header">
        <div class="dashboard-title">🛒 E-Commerce Sales Intelligence Dashboard</div>
        <div class="dashboard-subtitle">Real-time analytics with advanced SQL querying and interactive filtering</div>
    </div>
    """
    display(HTML(header_html))

    # Parse date range
    start_date, end_date = date_range if date_range else (None, None)

    # Display current filters
    filters_html = f"""
    <div style="background: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px;">
        <strong>Active Filters:</strong>
        Region: <span style="color: #667eea; font-weight: bold;">{region}</span> |
        Category: <span style="color: #667eea; font-weight: bold;">{category}</span> |
        Date Range: <span style="color: #667eea; font-weight: bold;">{start_date or 'All'} to {end_date or 'All'}</span>
    </div>
    """
    display(HTML(filters_html))

    # Fetch filtered data
    print("⏳ Loading data...")
    data = get_filtered_data(conn, region, category, start_date, end_date)
    print("✓ Data loaded successfully\n")

    # Display KPI cards
    create_kpi_cards(data)

    # Row 1: Heatmap and Category Pie
    print("\n📊 REGIONAL ANALYSIS\n" + "="*50)
    create_regional_heatmap(data)
    create_category_pie(data)

    # Row 2: Revenue Trends
    print("\n📈 REVENUE TRENDS\n" + "="*50)
    create_revenue_trends(data)

    # Row 3: Customer Analysis
    print("\n👥 CUSTOMER ANALYSIS\n" + "="*50)
    create_clv_chart(data)

    # Row 4: Product Performance
    print("\n⭐ PRODUCT PERFORMANCE\n" + "="*50)
    create_product_scatter(data)

    # Row 5: Top Categories (Window Function)
    print("\n🏆 ADVANCED SQL ANALYSIS\n" + "="*50)
    create_top_categories_table(data)

if conn and filter_options:
    # Create interactive widgets
    region_dropdown = widgets.Dropdown(
        options=filter_options['regions'],
        value='All',
        description='Region:',
        style={'description_width': '100px'}
    )

    category_dropdown = widgets.Dropdown(
        options=filter_options['categories'],
        value='All',
        description='Category:',
        style={'description_width': '100px'}
    )

    date_picker = widgets.SelectionRangeSlider(
        options=pd.date_range(
            start=filter_options['min_date'],
            end=filter_options['max_date'],
            freq='W'
        ).strftime('%Y-%m-%d').tolist(),
        index=(0, -1),
        description='Date Range:',
        continuous_update=False,
        style={'description_width': '100px'},
        layout=widgets.Layout(width='600px')
    )

    # Create interactive output
    interactive_output = interactive(
        create_dashboard,
        region=region_dropdown,
        category=category_dropdown,
        date_range=date_picker
    )

    print("✓ Dashboard ready! Use the controls above to filter data.\n")
    display(interactive_output)
else:
    print("✗ Database connection failed. Please run the analysis notebook first.")

---

## Dashboard Features

### 🎯 Key Performance Indicators (KPIs)
- **Total Revenue**: Sum of all completed orders
- **Total Orders**: Count of completed transactions
- **Average Order Value**: Mean transaction value
- **Average CLV**: Mean customer lifetime value

### 📊 Visualizations

1. **Regional Sales Heatmap**: Shows revenue distribution across regions and product categories
2. **Revenue Distribution**: Pie chart showing category contribution to total revenue
3. **Daily Revenue Trends**: Line chart with 7-day moving average using window functions
4. **Customer Lifetime Value**: Top 15 customers ranked by total spending
5. **Product Performance**: Scatter plot correlating ratings with revenue
6. **Top Categories by Region**: Window function analysis using RANK()

### 🔧 Advanced SQL Features

- **Window Functions**: `RANK()`, `AVG() OVER()` for moving averages
- **Complex JOINs**: Multiple table joins (customers ⟕ orders ⟕ products ⟕ reviews)
- **Aggregations**: `SUM()`, `COUNT()`, `AVG()`, `MIN()`, `MAX()`
- **CTEs**: Common Table Expressions for query optimization
- **Subqueries**: Nested SELECT statements
- **PARTITION BY**: Regional grouping for window functions

---

## Technical Notes

**Database Location**: `../analysis/ecommerce.db`

**Dependencies**:
- SQLite3 (built-in)
- pandas
- plotly
- ipywidgets

**Performance**: All queries are optimized with indexes on frequently queried columns.

---

*Created for Database Management Course - Inha University*